<a href="https://colab.research.google.com/github/soudagarminhaz/learning-game/blob/main/LearningGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GenAI Interactive Learning Game - All Modes, Multiplayer, Sounds, Animations (Google Colab Compatible)

import ipywidgets as widgets
from IPython.display import display, clear_output, Audio, HTML
import random
def show_question():
    output.clear_output()
    qlist = state["questions"]

    if state["index"] >= len(qlist):
        show_result()
        return

    current = qlist[state["index"]]
    widgets_list = []

    # Player turn title
    player_title = widgets.HTML(f"<h3 style='text-align:center;'>🧑 Player {state['player']}'s Turn</h3>")
    widgets_list.append(player_title)

    if state["mode"] == "quiz":
        qtext = widgets.HTML(f"<h4>{current['question']}</h4>")
        widgets_list.append(qtext)

        btns = []
        for opt in current['options']:
            b = widgets.Button(description=opt, layout=widgets.Layout(width='auto'))
            b.on_click(lambda x, o=opt: check_answer(o))  # Fix lambda binding
            btns.append(b)

        btn_box = widgets.VBox(btns)
        widgets_list.append(btn_box)

    elif state["mode"] == "puzzle":
        qtext = widgets.HTML(f"<h4>{current['puzzle']}</h4>")
        answer = widgets.Text(placeholder="Type your answer")
        submit = widgets.Button(description="Submit", button_style='primary')
        submit.on_click(lambda x: check_answer(answer.value))
        widgets_list.extend([qtext, widgets.HBox([answer, submit])])

    elif state["mode"] == "riddle":
        qtext = widgets.HTML(f"<h4>{current['riddle']}</h4>")
        answer = widgets.Text(placeholder="Type your answer")
        submit = widgets.Button(description="Submit", button_style='primary')
        submit.on_click(lambda x: check_answer(answer.value))
        widgets_list.extend([qtext, widgets.HBox([answer, submit])])

    with output:
        display(widgets.VBox(widgets_list))


# ========== Data ==========
questions = [
    {"type": "quiz", "question": "What is the capital of France?", "options": ["Berlin", "Madrid", "Paris", "Rome"], "correct_answer": "Paris", "level": "easy", "explanation": "Paris is the capital of France."},
    {"type": "quiz", "question": "Who painted the Mona Lisa?", "options": ["Van Gogh", "Da Vinci", "Picasso", "Monet"], "correct_answer": "Da Vinci", "level": "easy", "explanation": "Leonardo da Vinci painted it."},
    {"type": "quiz", "question": "What is the largest planet?", "options": ["Earth", "Saturn", "Jupiter", "Neptune"], "correct_answer": "Jupiter", "level": "medium", "explanation": "Jupiter is the largest."},
    {"type": "quiz", "question": "Atomic number of Gold?", "options": ["72", "79", "82", "118"], "correct_answer": "79", "level": "hard", "explanation": "Gold has atomic number 79."}
]

puzzles = [
    {"type": "puzzle", "puzzle": "What has to be broken before you can use it?", "answer": "egg", "level": "easy", "explanation": "An egg must be broken."},
    {"type": "puzzle", "puzzle": "I speak without a mouth and hear without ears. What am I?", "answer": "echo", "level": "medium", "explanation": "It's an echo."},
    {"type": "puzzle", "puzzle": "What has keys but can't open locks?", "answer": "piano", "level": "hard", "explanation": "A piano has keys."}
]

riddles = [
    {"type": "riddle", "riddle": "What comes once in a minute, twice in a moment, but never in a thousand years?", "answer": "m", "level": "medium", "explanation": "The letter 'm'."},
    {"type": "riddle", "riddle": "The more you take, the more you leave behind. What are they?", "answer": "footsteps", "level": "hard", "explanation": "Footsteps."}
]

stories = {
    "sun": "The golden sun rose over the hills, bathing the world in light...",
    "moon": "The silver moon hung low in the sky as stars whispered secrets...",
    "ocean": "Waves crashed endlessly on the shore, hiding tales of the deep..."
}

# ========== Sounds ==========
def play_correct():
    display(Audio("https://www.soundjay.com/buttons/sounds/button-3.mp3", autoplay=True))

def play_wrong():
    display(Audio("https://www.soundjay.com/buttons/sounds/button-10.mp3", autoplay=True))


# ========== State ==========
state = {
    "mode": None,
    "level": "easy",
    "player": 1,
    "players": [0, 0],
    "index": 0,
    "questions": []
}

# ========== Widgets ==========
mode_select = widgets.ToggleButtons(options=['Quiz', 'Puzzle', 'Riddle', 'Story'], description='Mode:')
level_select = widgets.RadioButtons(options=['easy', 'medium', 'hard'], description='Level:')
start_btn = widgets.Button(description="Start", button_style='success', icon='play')
word_input = widgets.Text(placeholder='Try: sun, moon, ocean...', description='Story Word:')
next_btn = widgets.Button(description="Next", icon='arrow-right')
restart_btn = widgets.Button(description="Restart", button_style='danger')
output = widgets.Output()

# ========== Game Setup ==========
def setup_game(*args):
    state["mode"] = mode_select.value.lower()
    state["level"] = level_select.value
    state["index"] = 0
    state["player"] = 1
    state["players"] = [0, 0]

    if state["mode"] == "quiz":
        state["questions"] = [q for q in questions if q['level'] == state["level"]]
    elif state["mode"] == "puzzle":
        state["questions"] = [p for p in puzzles if p['level'] == state["level"]]
    elif state["mode"] == "riddle":
        state["questions"] = [r for r in riddles if r['level'] == state["level"]]

    random.shuffle(state["questions"])
    show_question()

# ========== Show Question ==========
def show_question():
    output.clear_output()
    qlist = state["questions"]

    if state["index"] >= len(qlist):
        show_result()
        return

    current = qlist[state["index"]]
    with output:
        display(HTML(f"""
        <h2 style='text-align:center'>👤 Player {state['player']}'s Turn</h2>
        <div style='animation: fadein 1s;'>"""))

        if state["mode"] == "quiz":
            display(widgets.HTML(f"<h3>{current['question']}</h3>"))
            btns = []
            for opt in current['options']:
                b = widgets.Button(description=opt)
                b.on_click(lambda x, o=opt: check_answer(o))
                btns.append(b)
            display(widgets.VBox(btns))

        elif state["mode"] in ["puzzle", "riddle"]:
            key = "puzzle" if state["mode"] == "puzzle" else "riddle"
            display(widgets.HTML(f"<h3>{current[key]}</h3>"))
            answer = widgets.Text(placeholder="Type your answer")
            submit = widgets.Button(description="Submit")
            submit.on_click(lambda x: check_answer(answer.value))
            display(widgets.HBox([answer, submit]))

# ========== Check Answer ==========
def check_answer(ans):
    current = state["questions"][state["index"]]
    correct = (ans.lower().strip() == current.get("correct_answer", current.get("answer", "")).lower())

    output.clear_output()
    with output:
        if correct:
            state['players'][state['player'] - 1] += 1
            play_correct()
            display(HTML(f"<p style='color:green'><b>✅ Correct!</b> {current['explanation']}</p>"))
        else:
            play_wrong()
            display(HTML(f"<p style='color:red'><b>❌ Wrong!</b> {current['explanation']}</p>"))

        state['player'] = 2 if state['player'] == 1 else 1
        state['index'] += 1
        display(next_btn)

# ========== Next Question ==========
def next_q(x):
    show_question()

# ========== Result ==========
def show_result():
    output.clear_output()
    with output:
        p1, p2 = state['players']
        winner = "Draw" if p1 == p2 else f"Player {1 if p1 > p2 else 2} Wins"
        display(HTML(f"<h2>🎉 Game Over!</h2><h3>Scores</h3><p>Player 1: {p1}</p><p>Player 2: {p2}</p><h3>🏆 {winner}</h3>"))
        display(restart_btn)

# ========== Restart ==========
def restart(x):
    output.clear_output()
    display_setup()

# ========== Story Mode ==========
def start_story(*args):
    output.clear_output()
    word = word_input.value.strip().lower()
    story = stories.get(word)
    with output:
        if story:
            display(HTML(f"<h2>📖 Story: {word.capitalize()}</h2><p>{story}</p>"))
        else:
            display(HTML(f"<h3 style='color:red'>Story not found. Try: sun, moon, ocean</h3>"))
        display(restart_btn)

# ========== Setup UI ==========
def display_setup():
    ui = widgets.VBox([
        widgets.HTML("<h1 style='color: teal; text-align:center;'>🎮 GenAI Interactive Learning Game 🎮</h1>"),
        mode_select,
        widgets.HBox([level_select, word_input]),
        start_btn
    ])
    display(ui)

# ========== Events ==========
start_btn.on_click(lambda x: start_story() if mode_select.value == 'Story' else setup_game())
next_btn.on_click(next_q)
restart_btn.on_click(restart)

# ========== Run ==========
display_setup()
display(output)

# ========== CSS Animations ==========
display(HTML("""
<style>
@keyframes fadein {
  from {opacity: 0;}
  to {opacity: 1;}
}
div { animation: fadein 1s; }
</style>
"""))

Output()